In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
import xarray as xr
from geovoronoi import voronoi_regions_from_coords

import json
import os
from tqdm import tqdm

In [2]:
# make sure you have the lightlight rasters for PH first

ph_2019_nightlight_filepath = "output/clipped_ph_nl_median_masked_2019.tif"
ph_2020_nightlight_filepath = "output/clipped_ph_nl_median_masked_2020.tif"

## What this notebook is about

This notebook aggregates the raster per barangay in the Philippines. It makes the data easier to handle from a raster to a tabular format.

# Importing Brgy Data

The barangay files were obtained from [this source](https://github.com/faeldon/philippines-json-maps/tree/master/topojson/barangays/hires)

In [3]:
# write brgy to geojson
brgy_gdf = gpd.read_file('output/ph_brgys.geojson')
brgy_gdf.head()

,ADM1_PCODE,ADM1_EN,ADM2_PCODE,ADM2_EN,ADM3_PCODE,ADM3_EN,ADM4_PCODE,ADM4_EN,geometry
0,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602001,Angeles,"POLYGON ((121.99440 14.11326, 121.99405 14.112..."
1,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602002,Bacong,"POLYGON ((122.03987 14.14286, 122.03882 14.142..."
2,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602003,Balungay,"POLYGON ((122.05585 14.08601, 122.05575 14.085..."
3,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602004,Buenavista,"POLYGON ((122.07112 14.05975, 122.07174 14.059..."
4,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602005,Caglate,"POLYGON ((122.02153 14.15299, 122.02175 14.149..."


In [4]:
# these columns will serve as the primary key
brgy_cols = [col for col in brgy_gdf.columns if col != "geometry"]
brgy_cols

['ADM1_PCODE',
 'ADM1_EN',
 'ADM2_PCODE',
 'ADM2_EN',
 'ADM3_PCODE',
 'ADM3_EN',
 'ADM4_PCODE',
 'ADM4_EN']

## Import National Boundaries

In [5]:
ph_admin_filepath = "data/country.topo.0.1.json.txt"
ph_gdf = gpd.read_file(ph_admin_filepath).set_crs("epsg:4326")
ph_gdf

,id,Shape_Leng,Shape_Area,ADM0_EN,ADM0_PCODE,ADM0_REF,ADM0ALT1EN,ADM0ALT2EN,date,validOn,validTo,geometry
0,None,390.774375,24.538857,Philippines (the),PH,,,,2018-01-26T00:00:00+00:00,2018-01-30T00:00:00+00:00,1899-11-30T00:00:00+00:00,"MULTIPOLYGON (((119.46876 4.59360, 119.46907 4..."


# Aggregating 2019 Nightlight

In [6]:
%%time
nl_2019 = xr.open_rasterio(ph_2019_nightlight_filepath)
nl_2019

CPU times: user 66.6 ms, sys: 26.5 ms, total: 93.2 ms
Wall time: 248 ms


<xarray.DataArray (band: 1, y: 3969, x: 2959)>
[11744271 values with dtype=float32]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 21.12 21.12 21.11 21.11 21.1 ... 4.6 4.596 4.592 4.587
  * x        (x) float64 114.3 114.3 114.3 114.3 ... 126.6 126.6 126.6 126.6
Attributes:
    transform:      (0.0041666667, 0.0, 114.27708568755, 0.0, -0.004166666699...
    crs:            +init=epsg:4326
    res:            (0.0041666667, 0.004166666699999999)
    is_tiled:       0
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area

In [7]:
# convert zeros to nulls and drop them
# also convert xarray Dataset to geopandas GeoDataFrame
nl_2019 = nl_2019.where(nl_2019 > 0, drop = True)\
                .to_dataset(name = "nl_2019")\
                .to_dataframe()\
                .dropna()\
                .reset_index()\
                .drop(columns = "band")

nl_2019 = gpd.GeoDataFrame(nl_2019["nl_2019"], geometry = gpd.points_from_xy(nl_2019["x"], nl_2019["y"]),
                          crs = "epsg:4326")
nl_2019.head()

,nl_2019,geometry
0,0.116556,POINT (114.27917 9.72083)
1,0.153847,POINT (114.27917 9.71667)
2,0.179334,POINT (114.27917 9.71250)
3,0.132800,POINT (114.27917 9.70833)
4,0.783659,POINT (114.27917 5.92083)


In [8]:
%%time
# find the 
brgy_nl_2019 = gpd.sjoin(brgy_gdf, nl_2019, how = "left").drop(columns = "index_right")
brgy_nl_2019.head()

CPU times: user 15.6 s, sys: 279 ms, total: 15.9 s
Wall time: 18.4 s


,ADM1_PCODE,ADM1_EN,ADM2_PCODE,ADM2_EN,ADM3_PCODE,ADM3_EN,ADM4_PCODE,ADM4_EN,geometry,nl_2019
0,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602001,Angeles,"POLYGON ((121.99440 14.11326, 121.99405 14.112...",NaN
1,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602002,Bacong,"POLYGON ((122.03987 14.14286, 122.03882 14.142...",0.229397
2,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602003,Balungay,"POLYGON ((122.05585 14.08601, 122.05575 14.085...",NaN
3,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602004,Buenavista,"POLYGON ((122.07112 14.05975, 122.07174 14.059...",NaN
4,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602005,Caglate,"POLYGON ((122.02153 14.15299, 122.02175 14.149...",NaN


In [9]:
agg_func_dict = {
    "mean_nl_2019":"mean",
    "sum_nl_2019":"sum",
    "count_nl_2019":"count",
}

In [10]:
brgy_nl_2019 = brgy_nl_2019.groupby(brgy_cols)["nl_2019"]\
                            .agg(**agg_func_dict)\
                            .reset_index()\
                            .sort_values(by = "mean_nl_2019", ascending = False)
brgy_nl_2019.head()

,ADM1_PCODE,ADM1_EN,ADM2_PCODE,ADM2_EN,ADM3_PCODE,ADM3_EN,ADM4_PCODE,ADM4_EN,mean_nl_2019,sum_nl_2019,count_nl_2019
34106,PH130000000,NATIONAL CAPITAL REGION (NCR),PH137600000,"NCR, FOURTH DISTRICT (Not a Province)",PH137602000,CITY OF MAKATI,PH137602028,South Cembo,105.159500,105.159500,1
32778,PH130000000,NATIONAL CAPITAL REGION (NCR),PH133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",PH133901000,TONDO I / II,PH133901118,Barangay 118,102.748421,102.748421,1
32773,PH130000000,NATIONAL CAPITAL REGION (NCR),PH133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",PH133901000,TONDO I / II,PH133901110,Barangay 110,100.615196,100.615196,1
34136,PH130000000,NATIONAL CAPITAL REGION (NCR),PH137600000,"NCR, FOURTH DISTRICT (Not a Province)",PH137604000,CITY OF PARAÑAQUE,PH137604016,Vitalez,96.111870,96.111870,1
34099,PH130000000,NATIONAL CAPITAL REGION (NCR),PH137600000,"NCR, FOURTH DISTRICT (Not a Province)",PH137602000,CITY OF MAKATI,PH137602021,Post Proper Northside,89.100136,891.001404,10


# Aggregating 2020 Nightlight

Same process as 2019 nightlight

In [11]:
%%time
nl_2020 = xr.open_rasterio(ph_2020_nightlight_filepath)
nl_2020

CPU times: user 3.33 ms, sys: 1.72 ms, total: 5.04 ms
Wall time: 7.22 ms


<xarray.DataArray (band: 1, y: 3969, x: 2959)>
[11744271 values with dtype=float32]
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 21.12 21.12 21.11 21.11 21.1 ... 4.6 4.596 4.592 4.587
  * x        (x) float64 114.3 114.3 114.3 114.3 ... 126.6 126.6 126.6 126.6
Attributes:
    transform:      (0.0041666667, 0.0, 114.27708568755, 0.0, -0.004166666699...
    crs:            +init=epsg:4326
    res:            (0.0041666667, 0.004166666699999999)
    is_tiled:       0
    nodatavals:     (nan,)
    scales:         (1.0,)
    offsets:        (0.0,)
    AREA_OR_POINT:  Area

In [12]:
# convert zeros to nulls and drop them
# also convert xarray Dataset to geopandas GeoDataFrame
nl_2020 = nl_2020.where(nl_2020 > 0, drop = True)\
                .to_dataset(name = "nl_2020")\
                .to_dataframe()\
                .dropna()\
                .reset_index()\
                .drop(columns = "band")

nl_2020 = gpd.GeoDataFrame(nl_2020["nl_2020"], geometry = gpd.points_from_xy(nl_2020["x"], nl_2020["y"]),
                          crs = "epsg:4326")
nl_2020.head()

,nl_2020,geometry
0,0.612654,POINT (114.27917 5.89583)
1,0.858103,POINT (114.27917 5.89167)
2,0.439139,POINT (114.27917 5.88750)
3,0.947517,POINT (114.27917 5.88333)
4,1.036812,POINT (114.27917 5.87917)


In [13]:
%%time
brgy_nl_2020 = gpd.sjoin(brgy_gdf, nl_2020, how = "left").drop(columns = "index_right")
brgy_nl_2020.head()

CPU times: user 13.6 s, sys: 216 ms, total: 13.9 s
Wall time: 14.6 s


,ADM1_PCODE,ADM1_EN,ADM2_PCODE,ADM2_EN,ADM3_PCODE,ADM3_EN,ADM4_PCODE,ADM4_EN,geometry,nl_2020
0,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602001,Angeles,"POLYGON ((121.99440 14.11326, 121.99405 14.112...",NaN
1,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602002,Bacong,"POLYGON ((122.03987 14.14286, 122.03882 14.142...",0.250377
2,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602003,Balungay,"POLYGON ((122.05585 14.08601, 122.05575 14.085...",NaN
3,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602004,Buenavista,"POLYGON ((122.07112 14.05975, 122.07174 14.059...",NaN
4,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602005,Caglate,"POLYGON ((122.02153 14.15299, 122.02175 14.149...",NaN


In [14]:
agg_func_dict = {
    "mean_nl_2020":"mean",
    "sum_nl_2020":"sum",
    "count_nl_2020":"count",
}

In [15]:
brgy_nl_2020 = brgy_nl_2020.groupby(brgy_cols)["nl_2020"]\
                            .agg(**agg_func_dict)\
                            .reset_index()\
                            .sort_values(by = "mean_nl_2020", ascending = False)
brgy_nl_2020.head()

,ADM1_PCODE,ADM1_EN,ADM2_PCODE,ADM2_EN,ADM3_PCODE,ADM3_EN,ADM4_PCODE,ADM4_EN,mean_nl_2020,sum_nl_2020,count_nl_2020
32778,PH130000000,NATIONAL CAPITAL REGION (NCR),PH133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",PH133901000,TONDO I / II,PH133901118,Barangay 118,96.675514,96.675514,1
34136,PH130000000,NATIONAL CAPITAL REGION (NCR),PH137600000,"NCR, FOURTH DISTRICT (Not a Province)",PH137604000,CITY OF PARAÑAQUE,PH137604016,Vitalez,96.164757,96.164757,1
32773,PH130000000,NATIONAL CAPITAL REGION (NCR),PH133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",PH133901000,TONDO I / II,PH133901110,Barangay 110,93.569145,93.569145,1
32689,PH130000000,NATIONAL CAPITAL REGION (NCR),PH133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",PH133901000,TONDO I / II,PH133901020,Barangay 20,93.351723,653.462097,7
33464,PH130000000,NATIONAL CAPITAL REGION (NCR),PH133900000,"NCR, CITY OF MANILA, FIRST DISTRICT (Not a Pro...",PH133913000,PORT AREA,PH133913002,Barangay 650,86.792252,86.792252,1


## Concatenating 2019 and 2020 Nightlight in one table

In [16]:
brgy_nl = pd.merge(left = brgy_gdf, right = brgy_nl_2020, on = brgy_cols, how = "left", validate = "one_to_one")
brgy_nl = pd.merge(left = brgy_nl, right = brgy_nl_2019, on = brgy_cols, how = "left", validate = "one_to_one")
brgy_nl = brgy_nl.drop(columns = "geometry")
brgy_nl.head()

,ADM1_PCODE,ADM1_EN,ADM2_PCODE,ADM2_EN,ADM3_PCODE,ADM3_EN,ADM4_PCODE,ADM4_EN,mean_nl_2020,sum_nl_2020,count_nl_2020,mean_nl_2019,sum_nl_2019,count_nl_2019
0,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602001,Angeles,NaN,0.000000,0.0,NaN,0.000000,0.0
1,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602002,Bacong,0.250377,0.250377,1.0,0.229397,0.229397,1.0
2,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602003,Balungay,NaN,0.000000,0.0,NaN,0.000000,0.0
3,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602004,Buenavista,NaN,0.000000,0.0,NaN,0.000000,0.0
4,PH040000000,REGION IV-A (CALABARZON),PH045600000,QUEZON,PH045602000,ALABAT,PH045602005,Caglate,NaN,0.000000,0.0,NaN,0.000000,0.0


In [17]:
brgy_nl.to_csv("output/nl_2019_2020_per_brgy.csv", index = False)